In [1]:
import numpy as np
import transformers
import torch
import transformers as tf
import shutil
import urllib.request
import tarfile
from transformers import BertModel, BertTokenizer
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# Load các pretrain model
model = BertModel.from_pretrained('bert-base-uncased')
# Load model tu BERT pretrain. Cu the la model bert-base (bản nhỏ) và uncased (không care viết hoa, thường)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# load cái Tokenizer. Sử dụng để tokenize các từ trong câu.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
# Hàm chuẩn hoá câu
def standardize_data(row):
    # Xóa dấu chấm, phẩy, hỏi ở cuối câu
    row = re.sub(r"[\.,\?]+$-", "", row)
    # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace(""", " ") \
        .replace(":", " ").replace(""", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ") \
        .replace("<br />", " ").replace("?", " ")
    row = row.strip().lower()
    return row

In [5]:
def generate_batches(text, label, batch_size):
    for i in range(0, len(text), batch_size):
        yield text[i:i+batch_size], label[i:i+batch_size]

In [6]:
def load_data(batch_size):
    v_text = []
    v_label = []

    with open('/content/gdrive/MyDrive/Deep_Learning/train184.csv', encoding='latin-1') as f:
        lines = f.readlines()
        lines = lines[:1999]

    for line in lines:
        line = line.replace("\n","")
        v_text.append(standardize_data(line[:-2]))
        v_label.append(int(line[-1:].replace("\n", "")))

    print(v_label)
    return v_text, v_label

In [7]:
def make_bert_features(v_text, batch_size):
    v_tokenized = []
    max_len = 512

    # Tokenize các câu và thêm padding
    for i_text in v_text:
        line = tokenizer.encode(i_text, max_length=max_len, truncation=True, padding='max_length')
        v_tokenized.append(line)

    # Chuyển thành tensor
    input_ids = torch.tensor(v_tokenized).to(torch.long)
    attention_mask = torch.where(input_ids == tokenizer.pad_token_id, 0, 1)

    # Lấy features đầu ra từ BERT
    with torch.no_grad():
        last_hidden_states = model(input_ids=input_ids, attention_mask=attention_mask)[0]

    v_features = last_hidden_states[:, -2, :].numpy()
    return v_features

In [8]:
import re
re.compile('<title>(.*)</title>')

re.compile(r'<title>(.*)</title>', re.UNICODE)

In [9]:
batch_size=4
print("Chuẩn bị load dữ liệu....")
text, label = load_data(batch_size)
print("Đã load dữ liệu xong")

Chuẩn bị load dữ liệu....
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [10]:
print("Chuẩn bị tạo features từ BERT.....")
batch_generator = generate_batches(text, label, batch_size)
all_features = []
for batch_text, batch_label in batch_generator:
    batch_features = make_bert_features(batch_text, batch_size)
    #print(f"Processing batch {i} out of {len(batch_generator)}")
    all_features.append(batch_features)
features = np.concatenate(all_features, axis=0)

Chuẩn bị tạo features từ BERT.....


In [11]:
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(features, label, test_size=0.3, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.3, random_state=0)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [50]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(768,)))  # Lớp ẩn 1 với 128 nơron
model.add(Dense(64, activation='relu'))  # Lớp ẩn 2 với 64 nơron
model.add(Dense(32, activation='relu'))  # Lớp ẩn 3 với 32 nơron
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='softmax'))  # Lớp đầu ra với 2 nơron

In [51]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
from tensorflow.keras.utils import to_categorical

In [16]:
y_train=to_categorical(y_train, num_classes=2)
y_val=to_categorical(y_val, num_classes=2)
y_test=to_categorical(y_test, num_classes=2)

In [52]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/50
31/31 [==============================] - 2s 15ms/step - loss: 0.6730 - accuracy: 0.6057 - val_loss: 0.6000 - val_accuracy: 0.7548
Epoch 2/50
31/31 [==============================] - 0s 8ms/step - loss: 0.5413 - accuracy: 0.7671 - val_loss: 0.5320 - val_accuracy: 0.7476
Epoch 3/50
31/31 [==============================] - 0s 11ms/step - loss: 0.4205 - accuracy: 0.8090 - val_loss: 0.5142 - val_accuracy: 0.7548
Epoch 4/50
31/31 [==============================] - 0s 11ms/step - loss: 0.3316 - accuracy: 0.8601 - val_loss: 0.5550 - val_accuracy: 0.7333
Epoch 5/50
31/31 [==============================] - 0s 10ms/step - loss: 0.2647 - accuracy: 0.8948 - val_loss: 0.5910 - val_accuracy: 0.7500
Epoch 6/50
31/31 [==============================] - 0s 10ms/step - loss: 0.2257 - accuracy: 0.9162 - val_loss: 0.6141 - val_accuracy: 0.7286
Epoch 7/50
31/31 [==============================] - 0s 10ms/step - loss: 0.1713 - accuracy: 0.9418 - val_loss: 0.7072 - val_accuracy: 0.7548
Epoch 8/50
31/

In [53]:
score = model.evaluate(X_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

19/19 [==============================] - 0s 2ms/step - loss: 1.9340 - accuracy: 0.7517
Test loss: 1.9340267181396484
Test accuracy: 0.7516666650772095
